In [31]:
import pandas as pd

In [32]:
condition_file = '/home/vera/Documents/Arbeit/CRS/PsychNER/model/experiments/pubmedbert_condition_20240912/checkpoint-792_psychedelic_study_relevant_predictions.csv'
substance_file = '/home/vera/Documents/Arbeit/CRS/PsychNER/model/experiments/pubmedbert_substances_20240902/checkpoint-440_psychedelic_study_relevant_predictions.csv'

cond_label_map = {
        "0": "Psychiatric condition",
        "1": "Depression",
        "2": "Anxiety",
        "3": "Post-traumatic stress disorder (PTSD)",
        "4": "Alcoholism",
        "5": "Other addictions (e.g. smoking)",
        "6": "Anorexia",
        "7": "Alzheimer’s disease",
        "8": "Non-Alzheimer dementia",
        "9": "Substance abuse",
        "10": "(Chronic) Pain",
        "11": "Palliative Setting",
        "12": "Recreational Drug Use",
        "13": "Healthy Participants"
}

subs_label_map = {
        "0": "Ketamine",
        "1": "S-Ketamine",
        "2": "R-Ketamine",
        "3": "MDMA",
        "4": "LSD",
        "5": "Psilocybin",
        "6": "Psychedelic mushrooms",
        "7": "Ayahuasca",
        "8": "DMT",
        "9": "5-MeO-DMT",
        "10": "Mescaline",
        "11": "Ibogaine",
        "12": "Salvinorin A",
        "13": "Combination Therapy",
        "14": "Analogue",
        "15": "Unknown"
    }

threshold = 0.1

In [33]:
file = '/home/vera/Documents/Arbeit/CRS/PsychNER/data/raw_data/asreview_dataset_all_Psychedelic Study.csv'
df = pd.read_csv(file)
print(len(df))
# get distribution of included column, including nan
print(df['included'].value_counts(dropna=False))

9645
included
NaN    3939
1.0    3336
0.0    2370
Name: count, dtype: int64


In [34]:
cond_df = pd.read_csv(condition_file)
subs_df = pd.read_csv(substance_file)

print(cond_df.head())
print(subs_df.head())

     id                                               text  \
0  6152  Ketamine Versus Midazolam for Recurrence of Su...   
1  8321  Acute effects of MDMA and LSD co-administratio...   
2  1264  Trial of Psilocybin versus Escitalopram for De...   
3  7738  Acute autonomic and psychotropic effects of LS...   
4  3640  Distinct acute effects of LSD, MDMA, and D-amp...   

                                   prediction  \
0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
2  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   

                                         probability label  
0  [0.869410514831543, 0.13024155795574188, 6.260...    []  
1  [0.00012427952606230974, 4.3928230297751725e-0...    []  
2  [0.707538902759552, 0.29201456904411316, 6.750...    []  
3  [0.00015922347665764391, 6.170682900119573e-05...    []  
4  [0.00013469682016875595,

Recalculate the labels with threshold = 0.1

In [35]:
from ast import literal_eval

threshold = 0.1

def update_predictions(df, threshold):
    for i, row in df.iterrows():
        probabilities = literal_eval(row['probability'])
        new_prediction = [1 if prob >= threshold else 0 for prob in probabilities]
        df.at[i, 'prediction'] = str(new_prediction)
        
update_predictions(cond_df, threshold)
update_predictions(subs_df, threshold)
print(cond_df.head())
print(subs_df.head())        

     id                                               text  \
0  6152  Ketamine Versus Midazolam for Recurrence of Su...   
1  8321  Acute effects of MDMA and LSD co-administratio...   
2  1264  Trial of Psilocybin versus Escitalopram for De...   
3  7738  Acute autonomic and psychotropic effects of LS...   
4  3640  Distinct acute effects of LSD, MDMA, and D-amp...   

                                   prediction  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
2  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   

                                         probability label  
0  [0.869410514831543, 0.13024155795574188, 6.260...    []  
1  [0.00012427952606230974, 4.3928230297751725e-0...    []  
2  [0.707538902759552, 0.29201456904411316, 6.750...    []  
3  [0.00015922347665764391, 6.170682900119573e-05...    []  
4  [0.00013469682016875595,

In [36]:
# join the two dataframes on id
print(len(cond_df))
print(len(subs_df))
df = pd.merge(cond_df, subs_df, on='id', how='inner')
print(len(df))
df.rename(columns={'prediction_x': 'condition_prediction', 'prediction_y': 'substance_prediction'}, inplace=True)
print(df.columns)

3336
3336
3336
Index(['id', 'text_x', 'condition_prediction', 'probability_x', 'label_x',
       'text_y', 'substance_prediction', 'probability_y', 'label_y'],
      dtype='object')


In [37]:
# Get frequency of combination of condition and substance
df['condition_prediction'] = df['condition_prediction'].apply(lambda x: literal_eval(x))
df['substance_prediction'] = df['substance_prediction'].apply(lambda x: literal_eval(x))
df['condition'] = df['condition_prediction'].apply(lambda x: [cond_label_map[str(i)] for i, pred in enumerate(x) if pred == 1])
df['substance'] = df['substance_prediction'].apply(lambda x: [subs_label_map[str(i)] for i, pred in enumerate(x) if pred == 1])
print(df.head())



     id                                             text_x  \
0  6152  Ketamine Versus Midazolam for Recurrence of Su...   
1  8321  Acute effects of MDMA and LSD co-administratio...   
2  1264  Trial of Psilocybin versus Escitalopram for De...   
3  7738  Acute autonomic and psychotropic effects of LS...   
4  3640  Distinct acute effects of LSD, MDMA, and D-amp...   

                         condition_prediction  \
0  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
2  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]   

                                       probability_x label_x  \
0  [0.869410514831543, 0.13024155795574188, 6.260...      []   
1  [0.00012427952606230974, 4.3928230297751725e-0...      []   
2  [0.707538902759552, 0.29201456904411316, 6.750...      []   
3  [0.00015922347665764391, 6.170682900119573e-05...      []   
4  [0.000134

In [38]:
combinations = []

for _, row in df.iterrows():
    for cond in row['condition']:
        for subs in row['substance']:
            combinations.append((cond, subs))
combinations_df = pd.DataFrame(combinations, columns=['Condition', 'Substance'])
            
combination_counts = combinations_df.value_counts().reset_index(name='Count')
combination_counts['Percentage'] = combination_counts['Count'] / len(df) * 100
# sort by count
combination_counts.sort_values(by='Count', ascending=False, inplace=True)
# print top 5
print(combination_counts.head())

               Condition Substance  Count  Percentage
0   Healthy Participants      MDMA    970   29.076739
1  Psychiatric condition  Ketamine    814   24.400480
2             Depression  Ketamine    626   18.764988
3  Recreational Drug Use      MDMA    412   12.350120
4   Healthy Participants  Ketamine    358   10.731415
